In [3]:
# Load preprocessed data
import os
import sys
sys.path.append('..')

from src.data import data_pipeline

data_file_path = os.path.join('..', 'data', 'train_v9rqX0R.csv')
X, y = data_pipeline(data_file_path)

c:\Users\miu3kor\OneDrive - Bosch Group\data_folder\Personal\practice\big_mart_sales_iii\notebooks\..\src\data.py:35: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["Item_Fat_Content"][data["Item_Fat_Content"] == key] = FAT_MAP[key]
c:\U

KeyError: 'Item_Identifier_1'

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()